In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# 가게 목록 (링크 포함된 것만)
store_urls = {
    "스노우폭스뱅뱅점": "https://map.naver.com/p/search/%EC%8A%A4%EB%85%B8%EC%9A%B0%ED%8F%AD%EC%8A%A4%20%EB%B1%85%EB%B1%85%EC%A0%90/place/36503902?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "맥도날드서초뱅뱅점": "https://map.naver.com/p/search/%EB%A7%A5%EB%8F%84%EB%82%A0%EB%93%9C%20%EC%84%9C%EC%B4%88%EB%B1%85%EB%B1%85%EC%A0%90/place/18777471?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "큐뮬러스": "https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "짜짜루": "https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "아티제강남유니온센터점": "https://map.naver.com/p/search/%EC%95%84%ED%8B%B0%EC%A0%9C%20%EA%B0%95%EB%82%A8%EC%9C%A0%EB%8B%88%EC%98%A8%EC%84%BC%ED%84%B0%EC%A0%90/place/1247547202?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "어가람": "https://map.naver.com/p/search/%EC%96%B4%EA%B0%80%EB%9E%8C/place/815591818?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "파리바게뜨뱅뱅사거리점": "https://map.naver.com/p/search/%ED%8C%8C%EB%A6%AC%EB%B0%94%EA%B2%8C%EB%9C%A8%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1153631969?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "파라노이드강남뱅뱅카페": "https://map.naver.com/p/search/%ED%8C%8C%EB%9D%BC%EB%85%B8%EC%9D%B4%EB%93%9C%20%EA%B0%95%EB%82%A8%EB%B1%85%EB%B1%85%20%EC%B9%B4%ED%8E%98/place/1946966999?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "아티제뱅뱅사거리점": "https://map.naver.com/p/search/%EC%95%84%ED%8B%B0%EC%A0%9C%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1794092937?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "만복회해산물뱅뱅사거리점": "https://map.naver.com/p/search/%EB%A7%8C%EB%B3%B5%ED%9A%8C%ED%95%B4%EC%82%B0%EB%AC%BC%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1022301266?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "해머스미스커피뱅뱅사거리점": "https://map.naver.com/p/search/%ED%95%B4%EB%A8%B8%EC%8A%A4%EB%AF%B8%EC%8A%A4%EC%BB%A4%ED%94%BC%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1485428420?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "월참치": "https://map.naver.com/p/search/%EC%9B%94%EC%B0%B8%EC%B9%98/place/1231140993?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "고가네철판불백": "https://map.naver.com/p/search/%EA%B3%A0%EA%B0%80%EB%84%A4%EC%B2%A0%ED%8C%90%EB%B6%88%EB%B0%B1/place/1020912680?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "곰작골나주곰탕역삼초교점": "https://map.naver.com/p/search/%EA%B3%B0%EC%9E%91%EA%B3%A8%EB%82%98%EC%A3%BC%EA%B3%B0%ED%83%95%20%EC%97%AD%EC%82%BC%EC%B4%88%EA%B5%90%EC%A0%90/place/1713528555?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "사람사는고깃집김일도뱅뱅점": "https://map.naver.com/p/search/%EC%82%AC%EB%9E%8C%EC%82%AC%EB%8A%94%EA%B3%A0%EA%B9%83%EC%A7%91%EA%B9%80%EC%9D%BC%EB%8F%84%20%EB%B1%85%EB%B1%85%EC%A0%90/place/1775396166?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "잇풀키토김밥샌드위치샐러드강남점": "https://map.naver.com/p/search/%EC%9E%87%ED%92%80%ED%82%A4%ED%86%A0%EA%B9%80%EB%B0%A5%20%EC%83%8C%EB%93%9C%EC%9C%84%EC%B9%98%20%EC%83%90%EB%9F%AC%EB%93%9C%20%EA%B0%95%EB%82%A8%EC%A0%90/place/1945206174?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "봉구스밥버거역삼삼일점": "https://map.naver.com/p/search/%EB%B4%89%EA%B5%AC%EC%8A%A4%EB%B0%A5%EB%B2%84%EA%B1%B0%20%EC%97%AD%EC%82%BC%EC%82%BC%EC%9D%BC%EC%A0%90/place/35150671?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "세로방": "https://map.naver.com/p/search/%EC%84%B8%EB%A1%9C%EB%B0%A9/place/20619334?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "소뭉집본점": "https://map.naver.com/p/search/%EC%86%8C%EB%AD%89%EC%A7%91%20%EB%B3%B8%EC%A0%90/place/1810726526?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "리첸": "https://map.naver.com/p/search/%EB%A6%AC%EC%B2%B8/place/1598283551?c=13.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "지우네": "https://map.naver.com/p/search/%EC%A7%80%EC%9A%B0%EB%84%A4/place/513798334?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "중화카츠": "https://map.naver.com/p/search/%EC%A4%91%ED%99%94%EC%B9%B4%EC%B8%A0/place/1088907629?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "교토일식": "https://map.naver.com/p/search/%EA%B5%90%ED%86%A0%EC%9D%BC%EC%8B%9D/place/18584588?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "회춘식당": "https://map.naver.com/p/search/%ED%9A%8C%EC%B6%98%EC%8B%9D%EB%8B%B9/place/1982360091?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "노브랜드버거뱅뱅사거리점": "https://map.naver.com/p/search/%EB%85%B8%EB%B8%8C%EB%9E%9C%EB%93%9C%EB%B2%84%EA%B1%B0%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1138720229?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "라망드쉐프": "https://map.naver.com/p/search/%EB%9D%BC%EB%A7%9D%EB%93%9C%EC%89%90%ED%94%84/place/36324600?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "에슬로우커피뱅뱅사거리점": "https://map.naver.com/p/search/%EC%97%90%EC%8A%AC%EB%A1%9C%EC%9A%B0%EC%BB%A4%ED%94%BC%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/383636716?c=16.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "전주피순대": "https://map.naver.com/p/search/%EC%A0%84%EC%A3%BC%ED%94%BC%EC%88%9C%EB%8C%80/place/1201436770?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "오로스트커피": "https://map.naver.com/p/search/%EC%98%A4%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%BB%A4%ED%94%BC/place/1265136552?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "다슬음": "https://map.naver.com/p/search/%EB%8B%A4%EC%8A%AC%EC%9D%8C/place/1236915361?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "조순금닭도리탕": "https://map.naver.com/p/search/%EC%A1%B0%EC%88%9C%EA%B8%88%EB%8B%AD%EB%8F%84%EB%A6%AC%ED%83%95/place/1071467533?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "궁민김밥": "https://map.naver.com/p/search/%EA%B6%81%EB%AF%BC%EA%B9%80%EB%B0%A5/place/1352704000?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "더벤티뱅뱅사거리점": "https://map.naver.com/p/search/%EB%8D%94%EB%B2%A4%ED%8B%B0%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/572648886?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "돼지벅스": "https://map.naver.com/p/search/%EB%8F%BC%EC%A7%80%EB%B2%85%EC%8A%A4/place/1359583115?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "시원한대구탕": "https://map.naver.com/p/search/%EC%8B%9C%EC%9B%90%ED%95%9C%EB%8C%80%EA%B5%AC%ED%83%95/place/20916181?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "판문점부대찌개역삼점": "https://map.naver.com/p/search/%ED%8C%90%EB%AC%B8%EC%A0%90%EB%B6%80%EB%8C%80%EC%B0%8C%EA%B0%9C%20%EC%97%AD%EC%82%BC%EC%A0%90/place/18584748?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "청담배짱이": "https://map.naver.com/p/search/%EC%B2%AD%EB%8B%B4%EB%B0%B0%EC%A7%B1%EC%9D%B4/place/1776020426?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "통돼지두루치기김치찌개전문점역삼": "https://map.naver.com/p/search/%ED%86%B5%EB%8F%BC%EC%A7%80%EB%91%90%EB%A3%A8%EC%B9%98%EA%B8%B0%EA%B9%80%EC%B9%98%EC%B0%8C%EA%B0%9C%EC%A0%84%EB%AC%B8%EC%A0%90%20%EC%97%AD%EC%82%BC/place/35103360?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "봉평착한메밀": "https://map.naver.com/p/search/%EB%B4%89%ED%8F%89%EC%B0%A9%ED%95%9C%EB%A9%94%EB%B0%80/place/37491529?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "칸꼬시": "https://map.naver.com/p/search/%EC%B9%B8%EA%BC%AC%EC%8B%9C/place/1084480642?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "곰바로곰탕": "https://map.naver.com/p/search/%EA%B3%B0%EB%B0%94%EB%A1%9C%EA%B3%B0%ED%83%95/place/1148852390?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "문화시민": "https://map.naver.com/p/search/%EB%AC%B8%ED%99%94%EC%8B%9C%EB%AF%BC/place/1442991337?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "오늘은어떤닭": "https://map.naver.com/p/search/%EC%98%A4%EB%8A%98%EC%9D%80%20%EC%96%B4%EB%96%A4%EB%8B%AD/place/1854564675?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "카페블랑131": "https://map.naver.com/p/search/%EC%B9%B4%ED%8E%98%EB%B8%94%EB%9E%91131/place/1214592701?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "버드나무집서초본점": "https://map.naver.com/p/search/%EB%B2%84%EB%93%9C%EB%82%98%EB%AC%B4%EC%A7%91%20%EC%84%9C%EC%B4%88%EB%B3%B8%EC%A0%90/place/11627634?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "파스쿠찌뱅뱅사거리점": "https://map.naver.com/p/search/%ED%8C%8C%EC%8A%A4%EC%BF%A0%EC%B0%8C%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/33545753?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "남강매점": "https://map.naver.com/p/search/%EB%82%A8%EA%B0%95%EB%A7%A4%EC%A0%90/place/35112336?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "스타벅스서초태우빌딩점": "https://map.naver.com/p/search/%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4%20%EC%84%9C%EC%B4%88%ED%83%9C%EC%9A%B0%EB%B9%8C%EB%94%A9%EC%A0%90/place/1685227032?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "맥켄치킨": "https://map.naver.com/p/search/%EB%A7%A5%EC%BC%84%EC%B9%98%ED%82%A8/place/18360454?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "봉피양양재점": "https://map.naver.com/p/search/%EB%B4%89%ED%94%BC%EC%96%91%20%EC%96%91%EC%9E%AC%EC%A0%90/place/37997726?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "깡돈": "https://map.naver.com/p/search/%EA%B9%A1%EB%8F%88/place/37902481?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "북경반점": "https://map.naver.com/p/search/%EB%B6%81%EA%B2%BD%EB%B0%98%EC%A0%90/place/850010002?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "무화잠강남점": "https://map.naver.com/p/search/%20%EB%AC%B4%ED%99%94%EC%9E%A0%20%EA%B0%95%EB%82%A8%EC%A0%90/place/18783735?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "아워948": "https://map.naver.com/p/search/%20%EC%95%84%EC%9B%8C948/place/772273676?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "참족": "https://map.naver.com/p/search/%20%EC%B0%B8%EC%A1%B1/place/38226320?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "예당": "https://map.naver.com/p/search/%EC%98%88%EB%8B%B9%20%EB%8F%84%EA%B3%A1%EB%8F%99/place/18855701?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "다돈식당": "https://map.naver.com/p/search/%EB%8B%A4%EB%8F%88%EC%8B%9D%EB%8B%B9/place/1765708831?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "낙여삼": "https://map.naver.com/p/search/%EB%82%99%EC%97%AC%EC%82%BC/place/40774486?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "명동할머니국수뱅뱅사거리점": "https://map.naver.com/p/search/%EB%AA%85%EB%8F%99%ED%95%A0%EB%A8%B8%EB%8B%88%EA%B5%AD%EC%88%98%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/20642930?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "이천쌀밥": "https://map.naver.com/p/search/%EC%9D%B4%EC%B2%9C%EC%8C%80%EB%B0%A5%20%EA%B0%95%EB%82%A8/place/1814776214?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "또봉이통닭도곡점": "https://map.naver.com/p/search/%EB%98%90%EB%B4%89%EC%9D%B4%ED%86%B5%EB%8B%AD%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/1281659956?c=14.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "카린지린가네스낵바뱅뱅사거리점": "https://map.naver.com/p/search/%EC%B9%B4%EB%A6%B0%EC%A7%80%EB%A6%B0%EA%B0%80%EB%84%A4%EC%8A%A4%EB%82%B5%EB%B0%94%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1850658022?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "왕산골": "https://map.naver.com/p/search/%EC%99%95%EC%82%B0%EA%B3%A8/place/1164024954?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "서초남순남순대국본점": "https://map.naver.com/p/search/%EC%84%9C%EC%B4%88%EB%82%A8%EC%88%9C%EB%82%A8%EC%88%9C%EB%8C%80%EA%B5%AD%20%EB%B3%B8%EC%A0%90/place/11692888?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "브라운돈까스뱅뱅사거리점": "https://map.naver.com/p/search/%EB%B8%8C%EB%9D%BC%EC%9A%B4%EB%8F%88%EA%B9%8C%EC%8A%A4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1776021604?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "고품격커피공장뱅뱅사거리점": "https://map.naver.com/p/search/%EA%B3%A0%ED%92%88%EA%B2%A9%EC%BB%A4%ED%94%BC%EA%B3%B5%EC%9E%A5%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1710007700?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "두두돼지불백": "https://map.naver.com/p/search/%EB%91%90%EB%91%90%EB%8F%BC%EC%A7%80%EB%B6%88%EB%B0%B1/place/36690429?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "사천루": "https://map.naver.com/p/search/%EC%82%AC%EC%B2%9C%EB%A3%A8/place/1917137340?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "장수촌풍천장어": "https://map.naver.com/p/search/%EC%9E%A5%EC%88%98%EC%B4%8C%ED%92%8D%EC%B2%9C%EC%9E%A5%EC%96%B4/place/1997046045?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "뱅뱅막국수역삼본점": "https://map.naver.com/p/search/%EB%B1%85%EB%B1%85%EB%A7%89%EA%B5%AD%EC%88%98%20%EC%97%AD%EC%82%BC%EB%B3%B8%EC%A0%90/place/33964233?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "커피박스": "https://map.naver.com/p/search/%EC%BB%A4%ED%94%BC%EB%B0%95%EC%8A%A4/place/1982336630?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "뚜레쥬르뱅뱅사거리점": "https://map.naver.com/p/search/%EB%9A%9C%EB%A0%88%EC%A5%AC%EB%A5%B4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/18548937?c=15.45,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "에이림커피": "https://map.naver.com/p/search/%EC%97%90%EC%9D%B4%EB%A6%BC%EC%BB%A4%ED%94%BC/place/13349601?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "카페S": "https://map.naver.com/p/search/S%EC%B9%B4%ED%8E%98%20%EB%8F%84%EA%B3%A1%EB%8F%99/place/35781714?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "원할매이모네닭한마리": "https://map.naver.com/p/search/%EC%9B%90%ED%95%A0%EB%A7%A4%EC%9D%B4%EB%AA%A8%EB%84%A4%EB%8B%AD%ED%95%9C%EB%A7%88%EB%A6%AC/place/38325046?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "영이네추억의떡볶이": "https://map.naver.com/p/search/%EC%98%81%EC%9D%B4%EB%84%A4/place/1972614021?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "비채": "https://map.naver.com/p/search/%EB%B9%84%EC%B1%84/place/1729260818?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "한강수": "https://map.naver.com/p/search/%ED%95%9C%EA%B0%95%EC%88%98/place/13483118?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "고래똥": "https://map.naver.com/p/search/%EA%B3%A0%EB%9E%98%EB%98%A5/place/1035640949?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "두메도곡점": "https://map.naver.com/p/search/%EB%91%90%EB%A9%94%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/36932173?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "오하나도곡점": "https://map.naver.com/p/search/%EC%98%A4%ED%95%98%EB%82%98%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/1931511433?c=15.00,0,0,0,dh&isCorrectAnswer=true"
}

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# ChromeDriver 초기화 및 가게 정보 수집
for store_name, store_url in store_urls.items():
    try:
        # 네이버 지도 페이지로 이동
        driver.get(store_url)

        # entryIframe 로드 대기 및 전환
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

        # 가게 이름 검색
        store_name_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
        )
        store_name = store_name_element.text

        # 업종 카테고리 검색
        category_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
        )
        category = category_element.text

        # 별점 검색 (없으면 "없음")
        try:
            # 별점 가져오기
            rating_element = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
            )
            rating_text = rating_element.text
            rating = rating_text.replace("별점", "").strip()
        except:
            rating = "없음"

        # 데이터 저장
        store_info = {
            "가게 이름": [store_name],
            "업종 카테고리": [category],
            "별점": [rating],
        }
        df = pd.DataFrame(store_info)

        # 파일 이름 생성
        filename = f"C:/fintech_service/final_project/data/리뷰추출/업종정보/{store_name.replace(' ', '_')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"{filename} 저장 완료.")

    except Exception as e:
        print(f"{store_name} 정보 수집 중 오류 발생: {e}")



C:/fintech_service/final_project/data/리뷰추출/업종정보/스노우폭스_뱅뱅점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/맥도날드_서초뱅뱅점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/큐뮬러스.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/짜짜루.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/아티제_강남유니온센터점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/어가람.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/파리바게뜨_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/파라노이드_강남뱅뱅_카페.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/아티제_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/만복회해산물_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/해머스미스커피_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/월참치.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/고가네철판불백.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/곰작골나주곰탕_역삼초교점.csv 저장 완료.
C:/fintech_service/final_project/d

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# 가게 목록 (링크 포함된 것만)
store_urls = {
    "북경반점": "https://map.naver.com/p/search/%EB%B6%81%EA%B2%BD%EB%B0%98%EC%A0%90/place/850010002?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "무화잠강남점": "https://map.naver.com/p/search/%20%EB%AC%B4%ED%99%94%EC%9E%A0%20%EA%B0%95%EB%82%A8%EC%A0%90/place/18783735?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "아워948": "https://map.naver.com/p/search/%20%EC%95%84%EC%9B%8C948/place/772273676?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "참족": "https://map.naver.com/p/search/%20%EC%B0%B8%EC%A1%B1/place/38226320?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "예당": "https://map.naver.com/p/search/%EC%98%88%EB%8B%B9%20%EB%8F%84%EA%B3%A1%EB%8F%99/place/18855701?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "다돈식당": "https://map.naver.com/p/search/%EB%8B%A4%EB%8F%88%EC%8B%9D%EB%8B%B9/place/1765708831?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "낙여삼": "https://map.naver.com/p/search/%EB%82%99%EC%97%AC%EC%82%BC/place/40774486?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "명동할머니국수뱅뱅사거리점": "https://map.naver.com/p/search/%EB%AA%85%EB%8F%99%ED%95%A0%EB%A8%B8%EB%8B%88%EA%B5%AD%EC%88%98%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/20642930?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "이천쌀밥": "https://map.naver.com/p/search/%EC%9D%B4%EC%B2%9C%EC%8C%80%EB%B0%A5%20%EA%B0%95%EB%82%A8/place/1814776214?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "또봉이통닭도곡점": "https://map.naver.com/p/search/%EB%98%90%EB%B4%89%EC%9D%B4%ED%86%B5%EB%8B%AD%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/1281659956?c=14.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "카린지린가네스낵바뱅뱅사거리점": "https://map.naver.com/p/search/%EC%B9%B4%EB%A6%B0%EC%A7%80%EB%A6%B0%EA%B0%80%EB%84%A4%EC%8A%A4%EB%82%B5%EB%B0%94%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1850658022?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "왕산골": "https://map.naver.com/p/search/%EC%99%95%EC%82%B0%EA%B3%A8/place/1164024954?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "서초남순남순대국본점": "https://map.naver.com/p/search/%EC%84%9C%EC%B4%88%EB%82%A8%EC%88%9C%EB%82%A8%EC%88%9C%EB%8C%80%EA%B5%AD%20%EB%B3%B8%EC%A0%90/place/11692888?c=15.00,0,0,0,dh&isCorrectAnswer=true",
    "브라운돈까스뱅뱅사거리점": "https://map.naver.com/p/search/%EB%B8%8C%EB%9D%BC%EC%9A%B4%EB%8F%88%EA%B9%8C%EC%8A%A4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1776021604?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "고품격커피공장뱅뱅사거리점": "https://map.naver.com/p/search/%EA%B3%A0%ED%92%88%EA%B2%A9%EC%BB%A4%ED%94%BC%EA%B3%B5%EC%9E%A5%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1710007700?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp",
    "뚜레쥬르뱅뱅사거리점": "https://map.naver.com/p/search/%EB%9A%9C%EB%A0%88%EC%A5%AC%EB%A5%B4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/18548937?c=15.45,0,0,0,dh&placePath=%3Fentry%253Dbmp",
}

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# ChromeDriver 초기화 및 가게 정보 수집
for store_name, store_url in store_urls.items():
    try:
        # 네이버 지도 페이지로 이동
        driver.get(store_url)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "entryIframe")))


        # entryIframe 로드 대기 및 전환
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

        # 가게 이름 검색
        store_name_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
        )
        store_name = store_name_element.text

        # 업종 카테고리 검색
        category_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
        )
        category = category_element.text

        # 별점 검색 (없으면 "없음")
        try:
            # 별점 가져오기
            rating_element = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
            )
            rating_text = rating_element.text
            rating = rating_text.replace("별점", "").strip()
        except:
            rating = "없음"

        # 데이터 저장
        store_info = {
            "가게 이름": [store_name],
            "업종 카테고리": [category],
            "별점": [rating],
        }
        df = pd.DataFrame(store_info)

        # 파일 이름 생성
        filename = f"C:/fintech_service/final_project/data/리뷰추출/업종정보/{store_name.replace(' ', '_')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"{filename} 저장 완료.")

    except Exception as e:
        print(f"{store_name} 정보 수집 중 오류 발생: {e}")



C:/fintech_service/final_project/data/리뷰추출/업종정보/북경반점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/무화잠.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/OUR948.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/양재족발_참족.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/예당.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/다돈식당.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/낙여삼.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/명동할머니국수_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/이천쌀밥.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/또봉이통닭_서울도곡점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/카린지린가네스낵바_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/왕산골.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/남순남순대국_본점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/브라운돈까스_뱅뱅사거리점.csv 저장 완료.
C:/fintech_service/final_project/data/리뷰추출/업종정보/고품격커피공장_뱅뱅사거리

# 뚜레주르

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# 가게 목록 (링크 포함된 것만)
store_urls = {
    "뚜레쥬르뱅뱅사거리점": "https://map.naver.com/p/search/%EB%9A%9C%EB%A0%88%EC%A5%AC%EB%A5%B4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/18548937?c=15.45,0,0,0,dh&placePath=%3Fentry%253Dbmp",
}

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

for store_name, store_url in store_urls.items():
    try:
        # 네이버 지도 페이지로 이동
        driver.get(store_url)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "entryIframe")))

        # entryIframe 로드 대기 및 전환
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

        # 가게 이름 검색
        store_name_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='GMKzU']"))
        )
        store_name = store_name_element.text

        # 업종 카테고리 검색
        category_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='InNft']"))
        )
        category = category_element.text

        # 별점 검색 (없으면 "없음")
        try:
            # 별점 가져오기
            rating_element = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, "//span[@class='place_blind']"))
            )
            rating = rating_element.text.strip()
        except:
            rating = "없음"

        # 데이터 저장
        store_info = {
            "가게 이름": [store_name],
            "업종 카테고리": [category],
            "별점": [rating],
        }
        df = pd.DataFrame(store_info)

        # 파일 이름 생성
        filename = f"C:/fintech_service/final_project/data/리뷰추출/업종정보/{store_name.replace(' ', '_')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"{filename} 저장 완료.")

    except Exception as e:
        print(f"{store_name} 정보 수집 중 오류 발생: {e}")

# 드라이버 종료
# driver.quit()

뚜레쥬르뱅뱅사거리점 정보 수집 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x011633E3+25059]
	(No symbol) [0x010ECDE4]
	(No symbol) [0x00FCBEC3]
	(No symbol) [0x0100FD86]
	(No symbol) [0x0100FFCB]
	(No symbol) [0x0104D952]
	(No symbol) [0x01031F44]
	(No symbol) [0x0104B51E]
	(No symbol) [0x01031C96]
	(No symbol) [0x01003FAC]
	(No symbol) [0x01004F3D]
	GetHandleVerifier [0x01455543+3113795]
	GetHandleVerifier [0x0146A20A+3198986]
	GetHandleVerifier [0x014629E2+3168226]
	GetHandleVerifier [0x01203250+680016]
	(No symbol) [0x010F572D]
	(No symbol) [0x010F29D8]
	(No symbol) [0x010F2B75]
	(No symbol) [0x010E57D0]
	BaseThreadInitThunk [0x764CFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7742809E+286]
	RtlGetAppContainerNamedObjectPath [0x7742806E+238]



In [1]:
import pandas as pd

In [6]:
import json

# 텍스트 파일 읽기 및 딕셔너리 생성
input_file_path = "./정보추출.txt"
output_file_path = "./딕셔너리_저장.json"
data_dict = {}

try:
    with open(input_file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
        # 빈 줄 제거
        lines = [line.strip() for line in lines if line.strip()]
        
        # 데이터가 올바르게 2줄씩 있는지 확인
        if len(lines) % 2 != 0:
            raise ValueError("입력 파일의 줄 수가 홀수입니다. 가게 이름과 URL이 짝을 이루는지 확인하세요.")
        
        # 2줄씩 처리
        for i in range(0, len(lines), 2):
            key = lines[i]  # 가게 이름
            value = lines[i+1]  # URL
            data_dict[key] = value

    print("딕셔너리 생성 완료!")

    # 딕셔너리를 JSON 스타일로 저장
    with open(output_file_path, "w", encoding="utf-8") as file:
        json.dump(data_dict, file, ensure_ascii=False, indent=4)

    print(f"딕셔너리가 JSON 스타일로 저장되었습니다: {output_file_path}")

except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {input_file_path}")
except ValueError as e:
    print(f"데이터 처리 오류: {e}")
except Exception as e:
    print(f"예기치 못한 오류가 발생했습니다: {e}")


딕셔너리 생성 완료!
딕셔너리가 JSON 스타일로 저장되었습니다: ./딕셔너리_저장.json
